## Forecasting financial timeseries
### Introduction

Stock market time series are not easy to model. The dynamics of a stock market time series is often described as a random walk, meaning it is considered as a stochastic process, which means there may be no patterns in the time series to model.

Others suggest that the stock market is not necessarily a pure stochastic process, but that there may be some hidden pattern that a machine learning algorithm could be trained to uncover.

In this lesson, we will make the assumption that we can accurately model stock prices, so that we can decide when to buy or sell a particular stock. Time series modelling can help us achieve this aim.

We will start by looking over some basic techniques for modelling time series data before exploring a LSTM model. This will allow us to start simple and build up, as well as, set the context on why stock market timeseries is complex to model.

First, objective task is to select a machine learning model that can take into account the history of a sequence of data and then predict future values of the sequence over a period of time in the future that we specify. We will look at some simple methods, before moving on to a more complex example. But first, we need some data!



### Financial timeseries data

There are several ways to obtain stock data.  You can download historic stock data from a repository or you can use one of the many stock APIs, which provide both historic and current data about a whole range of stocks. StockAPIs will require you to sign up for an API key in order to use their service. You will also have a limit on the amount of data you can obtain.  Some potential sources of data include:

- *Polygon.io*: https://polygon.io

- *Yahoo Finance*: https://finance.yahoo.com

There are some limitations when using stockAPIs, particularly on the free tier.  We need a lot of data to be able identify both long term and short term patterns, and these APIs may not provide the access to the volume of data we would need. Therefore we will download a prepared dataset for our experiment.


### Install Python Libraries

In [ ]:
!pip install tensorflow pandas numpy matplotlib scikit-learn

### Loading the data
In this case study, we have downloaded a dataset of *IBM* stock prices covering the period 1962 to 2017.

The dataset is a historical record of daily market quotes, spanning from 1962 onwards. Each row represents one trading day, identified by the `Date` column in YYYY-MM-DD format. For example, 1962-01-02 indicates the first trading day of 1962. Because markets are closed at weekends and on bank holidays, successive dates may skip days; for instance, the entry after 1962-01-12 is 1962-01-15, reflecting the weekend gap. Chronological ordering is essential: downstream modelling relies on knowing that row 1's values precede row 2 in time.

Following the date, there are four price fields: `Open`, `High`, `Low`, and `Close`:

- `Open` is the price at which trading began that day; on 1962-01-02 it was 6.413.

- `High` is the maximum price reached during the session, which for 1962-01-02 also happens to be 6.413, indicating the price never rose above the opening.

- `Low` is the minimum price that day (6.3378 on 2 January 1962), and `Close` is the final price at which the security traded as the market closed (6.3378 on 2 January 1962).

These four fields allow us to gauge intraday volatility. For example, on 5 January 1962, the price opened at 6.3211, dipped to a low of 6.1958, and closed at 6.2041. Those intraday swings can be used to compute metrics like the daily range or to engineer features such as percentage changes between `Open` and `Close` or between `High` and `Low`.

- `Volume` column records the total number of contracts or shares traded over the course of the day. `Volume` is a core indicator of market liquidity; higher numbers typically signal heavier trading activity. For instance, on 8 January 1962, volume reached 655,676, which is substantially higher than the preceding day's 440,112, suggesting a surge of interest in that contract. Analysts often compare each day's trading volume against rolling averages to identify unusually active days, which could foreshadow breakouts or impending trend reversals.

- `OpenInt` (Open Interest) column is the count of outstanding contracts that have not yet been settled. In this particular extract, however, every entry in the `OpenInt` column is zero, implying that either the data source did not provide open interest for this contract or that open interest data wasn't available for that period. Although open interest can be a valuable gauge of how many participants maintain positions overnight, it is not strictly necessary for simple `Close`-price forecasting. As a result, most modelling exercises focus on the OHLCV (Open, High, Low, Close, Volume) fields and disregard the zeroed open interest.

Let's load the dataset. One important thing to do afterward we have loaded our data, is to ensure any time or date related columns are sorted chronologically:

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/martyn-harris-bbk/AppliedMachineLearning/refs/heads/main/data/ibm.us.txt')

# Sort DataFrame by date order
df = df.sort_values('Date')

df.head()

We will also set some parameters to matplotlib for when we plot the time series data as we work through our examples, so that everything is consistent:

In [ ]:
import matplotlib.pyplot as plt

# We will apply a clean, high‐contrast style to a Matplotlib plot.
# Our parameters update tick and axis colours, as well as, font sizes.


# Define parameters for tick and axis colours (black)
params = {
    "ytick.color": "k",       # y‐axis tick labels in black
    "xtick.color": "k",       # x‐axis tick labels in black
    "axes.labelcolor": "k",   # x/y label text in black
    "axes.edgecolor": "k"     # axis spines in black
}
# Update Matplotlib's runtime configuration with these parameters
plt.rcParams.update(params)


We visualise the daily closing prices over time. The plot draws directly from the `Close` column of our dataset and maps it against time on the x-axis to visual how the stock's value has evolved.

Plotting this series is first step in any time-series analysis. It gives us an immediate sense of whether the data exhibits an overall upward or downward trend, whether it shows signs of periodic or seasonal behaviour, and where any sharp spikes or dips occur—perhaps due to market shocks, news events, or structural breaks. These features are difficult to spot in tabular form but are visually obvious on a line chart.

Importantly, visualising the raw `Close` price helps us determine whether the data requires further preprocessing before being passed into a forecasting model. For instance, if we see very strong trends, we may consider detrending; if we observe erratic swings, smoothing techniques like moving averages might help; and if we notice missing periods or outliers, we can take corrective steps before training. This plot, then, informs us about the asset's historical behaviour but also guides how we shape the modelling pipeline:


In [ ]:
plt.figure(figsize = (15,6))

df['Close'].plot(color='blue', lw=1, label="Close", alpha=1)

plt.xlabel('Time (t)',fontsize=12)
plt.ylabel('$ Close Price',fontsize=12)

plt.legend()

Next, we zoom in on a specific 1,000‐day window (indices 10,000–10,999) to inspect the intraday price ranges more closely. Plotting the `Low` values in red and the `High` values in green allows us to clearly see how each day's price oscillated between its minimum and maximum.

Overlaying the yellow `Mid` line, gives us the average of that day's low and high, and a smoothed centre point that often lies between the extremes.

Examining such a window helps reveal short-term volatility and any abrupt shifts in trading behaviour; for instance, days where the gap between low and high suddenly widens might signal important news events or market reactions. We can then quickly identify each series and appreciate the amplitude of daily price swings, and how the midpoint evolves relative to those bounds:

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(
    range(10000, 11000),               # X‐axis: indices from 10000 to 10999
    df['Low'][10000:11000],            # Y‐axis: the Low values for that slice
    color='red',                       # Plot in red to distinguish Low
    lw=1,
    label="Low",                       # Legend label for this series
    alpha=0.7                          # Transparent so overlaps are visible
)

plt.plot(
    range(10000, 11000),               # Same x‐axis window
    df['High'][10000:11000],           # Y‐axis: the High values for days 10000–10999
    color='green',                     # Plot in green to distinguish High
    lw=1,
    label="High",                      # Legend label for this series
    alpha=0.7
)

plt.plot(
    range(10000, 11000),               # Again, same x‐axis indices
    ((df['Low'] + df['High']) / 2.0)[10000:11000],  # Midpoint: average of Low and High
    color='yellow',
    lw=1,
    label="Mid",                       # Legend label for the midpoint series
    alpha=0.7
)

plt.xlabel('Time (t)')
plt.ylabel('Prices ($)')

plt.legend()


### Extracting features

There are several attributes we could use when modelling stock prices—such as the daily closing price, which reflects the final value of the stock at the end of each trading session. However, rather than relying solely on the close, we aim to capture a more balanced view of the daily movement by using a *mid-price*.

The mid-price is calculated as the average of the day`s highest and lowest prices. This gives us a single feature that smooths out some of the daily noise and offers a clearer sense of the underlying trend. This mid-price will serve as the sole input feature for our model:


In [ ]:
# First we calculate the mid prices from the highest and lowest prices for the stock
high_prices = df.loc[:,'High'].to_numpy()
low_prices = df.loc[:,'Low'].to_numpy()

mid_prices = (high_prices + low_prices) * 0.5 # We take an average

### Resampling

To evaluate how well our model generalises, we need to divide the time series into training and testing sets. Unlike classification problems where we might use `train_test_split` from Scikit-learn to randomly partition features (`X`) and labels (`y`), forecasting problems require us to preserve the **temporal order** of the data. This is because future values must not leak into the training process.

In this context, we are predicting the next value in a sequence rather than classifying a static label. Therefore, we manually split the series, using the earlier portion for training and holding back the later portion for testing. This approach allows us to simulate how the model would behave in a real-world setting, where future data is unseen during training.

We use the mid-price sequence and reserve approximately 33% of it as test data. Computing a split index based on the total number of time steps (see code below) ensures the first 67% of the sequence is used to train the model, and the remaining 33% is kept for evaluation. This kind of chronological split respects the natural ordering of time-series data.

Below is the code that performs this division, along with print statements to confirm the size of each subset:


In [ ]:
# Determine the total number of mid-price observations
size = len(mid_prices)

# Define the proportion of data to reserve for testing (33%)
test_size = 0.33

# Compute the index at which to split the series:
# size * test_size gives the size of the test set;
# subtracting this from the total gives the split index
test_split = size - round(size * test_size)

# Slice the array to create the training set (first 67% of data)
train_data = mid_prices[:test_split]

# Slice the array to create the testing set (last 33% of data)
test_data = mid_prices[test_split:]

# Record the actual sizes of each set for reference
train_size = len(train_data)
test_size = len(test_data)

# Display the sizes to confirm the split
print(train_size)
print(test_size)

This ensures we build the model on past information and assess it on genuinely unseen future data, mimicking a real forecasting scenario.

### Preprocessing the Data for Neural Network Training

Neural networks are sensitive to the scale of input features. When training on raw price data, especially stock prices that can range widely over time, large input magnitudes can lead to unstable optimisation often causing issues such as vanishing or exploding gradients. This makes training slow or ineffective.

To address this, we apply *min–max* scaling to  rescale the data to lie within a consistent range (typically between 0 and 1). This ensures that all input values are treated proportionally, allowing the network to converge more reliably during training (hopefully).

We begin by reshaping the training and testing arrays into the 2D format expected by scikit-learn’s `MinMaxScaler`. Importantly, we fit the scaler *only* on the training set. This avoids *data leakage*, a situation where information from the test set influences the training process, leading to overly optimistic performance estimates. Once fitted, we apply the same transformation to the test data, ensuring both subsets are scaled consistently:

In [ ]:
import sklearn
from sklearn.preprocessing import MinMaxScaler

# Initialise the MinMaxScaler to normalise inputs to the range [0, 1]
scaler = MinMaxScaler()

# Reshape the data to 2D arrays (required by sklearn scalers)
train_data = train_data.reshape(-1, 1)
test_data = test_data.reshape(-1, 1)

# Fit the scaler on the training set only and transform it
train_data = scaler.fit_transform(train_data)

# Apply the same transformation to the test set
test_data = scaler.transform(test_data)

This step is essential for training an LSTM (or any deep learning model) on financial data, as it ensures stable gradient flow and efficient learning across epochs.

We now smooth the time series by applying a window size of 2000, and transform the values obtained from each sliding window. We select a large value as the window size, as the sliding window process can create a break at the end of each window, due to the fact that each window is normalised independently.  This means that there are points in our data that are affected by this process, causing a small loss in information, but there tends not to be enough data points affected for this to have a large impact on performance of the model.

We also reshape the training and test data back to the original row and column shape ready for the next step:

In [ ]:
# Train the MinMaxScaler with training data and smooth data
smoothing_window_size = 2000

# Generate a series of windows over the training data for smoothing (window size = 2000).
for di in range(0, 8000, smoothing_window_size):
    scaler.fit(train_data[di : di + smoothing_window_size, : ])
    train_data[di: di + smoothing_window_size, : ] = scaler.transform(train_data[di : di + smoothing_window_size, :])

# Normalise the remaining data
scaler.fit(train_data[di + smoothing_window_size: , :])
train_data[di + smoothing_window_size : , : ] = scaler.transform(train_data[di + smoothing_window_size: , :])

# Reshape both train and test to their original arrangement.
train_data = train_data.reshape(-1)

# Normalise test data using the scaler that we fit to the training data
test_data = scaler.transform(test_data).reshape(-1)

mid_prices

### Basic methods: One-Step Ahead Prediction

Before diving into complex models, it’s helpful to establish some baseline methods for forecasting. One of the simplest ways to predict the next time step in a stock price series is to use averaging techniques. These methods estimate the future price based on the historical values, without the need for training or tuning parameters.

You may have seen impressive examples where researchers apply deep learning models to predict exact future stock prices. But if precise forecasting were that easy, such models would be used to consistently beat the market, and their creators would likely be very wealthy. In reality, exact-value prediction in financial time series is highly uncertain and often impractical. Simple, interpretable methods can often perform just as well or better in short-term forecasts. Here, we explore two straightforward techniques:

- *Simple averaging*, where we take the mean of the last few observations, and
- *Exponential Moving Average (EMA)*, which gives more weight to recent prices.

We’ll apply each method to predict the next value in the series, one step at a time. For each prediction, we compute the squared difference from the actual next value—this gives us a *Mean Squared Error (MSE)* for each time step.

We will be collecting all these individual errors into an array, so that we can then compute the overall average MSE for each method. This gives us a quantitative basis for comparing the two approaches and helps us understand how well these naive predictors perform before we introduce more advanced models:


#### Simple average

First, we attempt to predict the stock market mid price for the next time step (`t+1`) as an average of the previously observed stock market prices within a fixed window size.  We will set the size of the window to a 30 day period representing a month:

In [ ]:
import numpy as np

# Set the window size for the averaging method — the number of past points to use for each prediction
window_size = 30

# Get the total number of observations in the training data
N = train_data.size

# Initialise empty lists to store predicted values, corresponding dates, and error terms
std_avg_predictions = []  # one-step-ahead predictions using simple average
std_avg_x = []            # x-axis labels (dates) for plotting
mse_errors = []           # squared error between predicted and actual values

# Loop through each time step starting from the end of the first full window
for pred_idx in range(window_size, N):

    # Select the date corresponding to this prediction index
    if pred_idx >= N:
        # (This clause will not trigger inside the loop, but is often used when forecasting beyond known data)
        date = dt.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        # For in-sample predictions, fetch the corresponding date from the original dataframe
        date = df.loc[pred_idx, 'Date']

    # Extract the previous `window_size` data points (e.g., previous 30 days)
    window_of_observations = train_data[pred_idx - window_size : pred_idx]

    # Calculate the mean of the window — our predicted value for the next time step
    avg = np.mean(window_of_observations)

    # Append the prediction to the list
    std_avg_predictions.append(avg)

    # Compute and store the squared error for this prediction
    mse_errors.append((avg - train_data[pred_idx]) ** 2)

    # Store the date for plotting
    std_avg_x.append(date)

# Compute and print the average Mean Squared Error, multiplied by 0.5 (optional, for comparison with half-MSE formulations)
print('MSE error for standard averaging: %.5f' % (0.5 * np.mean(mse_errors)))

import matplotlib.pyplot as plt

# Combine train and test data for full-series visualisation
full_data = np.concatenate([train_data, test_data], axis=0)

# Fit the scaler on the full data just for visualisation purposes (not used here directly)
scaler = MinMaxScaler(feature_range=(0, 1))
all_mid_data_scaled = scaler.fit_transform(full_data.reshape(-1, 1)).astype("float32")

plt.figure(figsize=(15, 6))

# Plot the true mid-price series (grey) across the full timeline
plt.plot(range(df.shape[0]), full_data, color='grey', label='True Mid-Price', lw=1, alpha=0.7)

# Overlay our predictions from the simple moving average (red)
# Start plotting from index = window_size since that's when predictions begin
plt.plot(range(window_size, N), std_avg_predictions, color='red', label='Simple Average Prediction', lw=1, alpha=0.8)

plt.xlabel('Time (t)')
plt.ylabel('Mid Price ($)')

plt.legend()

plt.title('One-Step ahead prediction using simple Averaging')

plt.show()

As we can see it follows the pattern of the stock price quite closely. The overall MSE suggests that the model performed quite well, but only if we want to obtain very short predictions into the future. This approach seems reasonable when we consider the fact that the stock price is unlikely to deviate too extremely over a 30 day period.

However, there are more sophisticated averaging techniques that we can explore in the form of the *Exponential Moving Average (EMA)*:

#### Exponential Moving Average (EMA)

The Exponential Moving Average (EMA) is a variation of the moving average that gives more weight to recent data points. While a simple moving average (SMA) treats all values in the window equally, EMA applies a decaying weight, so newer observations influence the result more than older ones.

This weighting makes the EMA more responsive to recent changes in stock price, which can be especially useful when tracking short-term trends or detecting reversals more quickly. Unlike the SMA, which smooths data evenly, the EMA reacts more sharply to recent movements while still preserving the smoothing benefits of averaging.

In our case, we apply an EMA over a 30-day window and use a smoothing factor of 0.8. This parameter controls how heavily recent values are weighted: a higher value (closer to 1) gives more influence to the most recent data, making the EMA even more sensitive to price fluctuations. A lower value (closer to 0) would behave more like a traditional moving average with a longer memory:

In [ ]:
# Smoothing factor for the exponential moving average (EMA)
# A value closer to 1 gives more weight to past EMA, a value closer to 0 gives more weight to the latest data point
weight = 0.8

# Total number of points in the training dataset
N = train_data.size

# Lists to store EMA predictions, corresponding x-values (dates), and squared errors
run_avg_predictions = []
run_avg_x = []
mse_errors = []

# Initialise the running mean (EMA) to 0 before starting
running_mean = 0.0

# The first EMA prediction (at index 0) is set to the initial running_mean (0.0)
run_avg_predictions.append(running_mean)

# Loop over each index in the training data to update the EMA and compute errors
for pred_idx in range(1, N):
    # Update the running mean (EMA) using the previous EMA and the last actual data point:
    #    EMA_t = weight * EMA_{t-1} + (1 - weight) * actual_{t-1}
    running_mean = running_mean * weight + (1.0 - weight) * train_data[pred_idx - 1]

    # Append the new EMA value as the prediction for the current index
    run_avg_predictions.append(running_mean)

    # Compute the squared error between the EMA prediction and the true value at this index
    mse_errors.append((run_avg_predictions[-1] - train_data[pred_idx]) ** 2)

    # Record the corresponding date (or index) for plotting—'date' must be defined earlier to align with pred_idx
    run_avg_x.append(date)

# After looping through all points, calculate and print the mean squared error (MSE) for the EMA predictions:
# We divide by 2 to match a half‐MSE convention (0.5 * mean squared error).
print('MSE error for EMA averaging: %.5f' % (0.5 * np.mean(mse_errors)))

# Plot the results
plt.figure(figsize = (15, 6))

plt.plot(range(df.shape[0]), full_data, color='grey', lw=1, label='True', alpha=0.7)
plt.plot(range(0,N), run_avg_predictions, color='red', lw=1, label='Prediction', alpha=0.5)

plt.xlabel('Time (t)')
plt.ylabel('Mid Price ($)')

plt.title('One-Step ahead prediction using EMA averaging')
plt.show()

After plotting the results, we can see that EMA fits a perfect line over the true distribution with a very low MSE suggesting the model is a good fit.

As we have seen, we have done a good job of predicting the next value of a stock. However, we are only limited to one-step ahead prediction. We would ideally like to model longer periods into the future than a single day in order to have more time to act should we wish to buy or sell stock.

In general, predicting the actual value of a stock at future points in time is not that useful for time series analysis.  We are more interested in summarising the trend of the stock price in the future. We would like to know whether it will go up or down in price over the next 30 days, for instance, but to do this we need to address the short-comings of these short-term predictions.

### LSTMs approach

LSTM models have dominated time series prediction recently, as they are well-suited to modelling sequential data that have both short and long range dependencies between data points in the past.  We can therefore use Long Short-Term Memory models to predict any number of time points into the future for our stock.

To recap from an earier lesson, an LSTM node (or cell) has 5 components, which allow it to model both long-term and short-term data points by storing a memory of past observations, which it keeps, updates, or discards using a number of different gates:

- *Cell state*: The internal memory which stores short and long-term memories
- *Hidden state*: The output state information calculated using the combinations of the current input, previous hidden state and current cell input which are used to predict the future stock prices. The hidden state selects between the short or long-term memory (sometimes both types) stored in the cell state to make the next prediction.
- *Input gate*: Selects how much information from the current input flows into the cell state
- *Forget gate*: Selects how much information from the current input and the previous cell state flows into the current cell state.
- *Output gate*: Decides on how much information from the current cell state flows into the hidden state, selecting between the long-term memories, or both short-term memories and long-term memories.

### Smoothing
Before training our LSTM model, we first smooth the input series using the Exponential Moving Average (EMA) introduced earlier. This step helps to reduce short-term noise—those sharp, erratic fluctuations in price—and instead emphasises the broader trend over time.

Smoothing the data makes it easier for the model to learn meaningful patterns without being distracted by minor, high-frequency variations that may not contribute to long-term forecasting performance.

In the following example, we apply this smoothing *only* on the training data, preserving the integrity of the test set for fair evaluation later on.

We first initialise the smoothed value (EMA) to zero and define a smoothing constant *gamma*. A smaller value of gamma (e.g. 0.1) smooths more heavily, meaning the model reacts slowly to new changes. A higher value (e.g. 0.8) would make the EMA more responsive to recent movements (as we discussed).

We then loop through the training data and recursively compute the smoothed series. Each new value is a weighted blend of the current raw input and the previously smoothed value. The result is a version of the training sequence with reduced noise, making it easier for the LSTM to model longer-term patterns.

Once smoothing is complete, we concatenate the modified training data with the untouched test set. This combined series is then scaled using `Min–Max` normalisation, mapping all values to the range [0, 1] to ensure we have stable gradient updates and to avoid issues caused by differing value magnitudes:

In [ ]:
# Apply exponential moving average smoothing to the training data.
# This helps reduce short-term volatility and sharp spikes.
EMA = 0.0             # Initialise the EMA value
gamma = 0.1           # Smoothing factor (controls the weight of recent values)

# Loop through the training set and apply recursive EMA smoothing
for ti in range(train_size):
    EMA = gamma * train_data[ti] + (1 - gamma) * EMA
    train_data[ti] = EMA            # Overwrite the value with its smoothed version

# Combine smoothed training data with raw test data
# (used later for generating a full scaled dataset for inference and plotting)
all_mid_data = np.concatenate([train_data, test_data], axis=0)

# Initialise the MinMaxScaler to normalise all values between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Apply scaling - reshape to 2D as required by sklearn
all_mid_data_scaled = scaler.fit_transform(
    all_mid_data.reshape(-1, 1)
).astype("float32")    # Keep data type consistent for compatibility with model inputs


Essentially we have the same data as the previous EMA example, but you can begin to copy from this point forth if you wish to experiment in a separate notebook later.

### The model

We now get to the best part, and set up an LSTM model to learn patterns in our stock price data. LSTM stands for *Long Short-Term Memory*, as we saw it's a type of neural network that's especially good at working with sequences, like time series.

The key idea behind an LSTM is that it doesn't just look at one point in time. Instead, it remembers what it saw before and uses that memory to make better predictions. At each step, the model takes in the stock price from the current day and also carries forward what it learned from previous days.

Before we train the model, we need to set a few key settings. These are our *hyperparameters*. First, we define `D`, which is the number of input features. In our case, we’re using just one: the mid-price of the stock.

Next is `num_unrollings`. This tells the model how many time steps to look back when it makes a prediction. Instead of learning from just one day at a time, we let the model learn from a sequence of, say, 50 days. The longer the sequence, the more context the model has, which often leads to better forecasts.

We also set the `batch_size`. This tells the model how many training samples to process at once. For example, if `batch_size = 32`, the model will train on 32 small sequences of stock prices in parallel.

Lastly, we define the number of neurons (or units) in each layer of the model, using the `num_nodes` parameter. These neurons are the model's internal processing units. In this setup, we use a deep LSTM with three layers: the first two layers have 200 units each, and the third has 150. More layers and neurons allow the model to learn more complex patterns, though they also require more data and time to train:

In [ ]:
import tensorflow as tf

D = 1                     # Dimensionality of the data (1-D series)
num_unrollings = 50       # Number of time steps
batch_size = 500          # Samples per batch

num_nodes = [200,200,150] # Hidden units in each of the 3 LSTM layers

n_layers = len(num_nodes) # 3 layers

dropout = 0.2             # Dropout rate

#### Inputs and outputs layers

Before we can train our LSTM model, we need to reshape our price data into a format that the network can learn from. LSTMs are designed to learn from *sequences*, so instead of feeding the model input layer a single price at a time, we give it short windows of consecutive prices and ask it to predict the next one in the sequence.

We do this using a technique called a *sliding window*. The function `make_windows()` breaks the data into overlapping sequences: for each window, it takes `num_unrollings` consecutive prices as the input (`X`) and the next price as the target output (`Y`). For example, if `window_size = 50`, the model sees the last 50 prices and tries to predict the 51st. The output `X_train` and `X_test` have three dimensions:

- *number of samples*: how many windows we created.
- *window size*: number of time steps in each input.
- *features*: just 1 in our case, the mid-price.

The labels `Y_train` and `Y_test` are 2D arrays, containing the price the model is trying to predict for each sequence. Finally, we wrap the training data into a TensorFlow `Dataset`. This is a high-performance pipeline that:

- shuffles the windows (so the model doesn’t learn the order explicitly),
- groups them into *batches* (so training runs faster),
- and prefetches data to keep the training loop efficient.

This preprocessing step ensures that the LSTM is trained on meaningful sequences of data, delivered in the right shape and format for learning time-dependent patterns in stock prices:

In [ ]:
# Build our own TensorFlow dataset for later use in training the LSTM
import numpy as np

# train_data and test_data are 1D NumPy arrays of prices (already smoothed and scaled earlier)
# We now convert these into supervised learning datasets: sequences of past values predicting the next value

def make_windows(data_array, window_size):
    # Converts a 1D time series into overlapping input/output pairs:
    # Each input X is a sequence of window_size consecutive prices
    # Each output Y is the value that immediately follows the sequence

    X, Y = [], []
    for i in range(len(data_array) - window_size):
        # Create one input window: [price_i, price_{i+1}, ..., price_{i+window_size-1}]
        X.append(data_array[i : i + window_size])

        # Store the next value after the window: price_{i+window_size}
        Y.append(data_array[i + window_size])

    # Reshape X to (samples, timesteps, features) as expected by LSTM
    X = np.array(X).reshape(-1, window_size, 1)

    # Reshape Y to (samples, 1) — each target is a single price value
    Y = np.array(Y).reshape(-1, 1)

    return X, Y

# Apply windowing to training and test sets using the same window size (num_unrollings)
X_train, Y_train = make_windows(train_data, num_unrollings)
X_test,  Y_test  = make_windows(test_data,  num_unrollings)

# Prepare the training dataset using TensorFlow's efficient Dataset API
train_dataset = (
    tf.data.Dataset
      .from_tensor_slices((X_train, Y_train))  # Wrap X and Y arrays into a tf.data.Dataset
      .shuffle(buffer_size=10000)              # Shuffle data to improve training stability
      .batch(batch_size, drop_remainder=True)  # Group samples into batches; discard leftover if incomplete
      .prefetch(tf.data.AUTOTUNE)              # Optimise performance by preloading batches during training
)

To recap, each sample in the dataset contains a small window of past prices used to predict the next price. The `tf.data.Dataset` pipeline enables efficient batching, shuffling, and preloading, ensuring smooth and scalable training with your LSTM model. This is also good, since you can easily pickle the dataset and load it back later.

To forecast the next stock price value, we construct a deep LSTM model with three stacked LSTM layers followed by a dense output layer. Each LSTM layer processes sequential price data and passes its output to the next layer, enabling the network to learn increasingly complex temporal patterns.

We begin by specifying the input shape: a sequence of num_unrollings time steps, each containing a single feature (the mid-price), which we define with a Keras Input layer. The data then flows through three LSTM layers, each configured with a specific number of hidden units (num_nodes). We apply dropout after each LSTM layer to reduce overfitting by randomly omitting a fraction of the units during training. This encourages the model to learn more robust features and generalise better to unseen data.

The final layer is a fully connected Dense layer that takes the output of the last LSTM cell and produces a single numeric prediction representing the estimated price at the next time step. This acts as a simple linear regression output on top of the learned sequence representation. The full model construction and compilation code, is shown below:

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense

# Define the input shape for the LSTM model
# Each input sequence has `num_unrollings` time steps, with `D` features per time step (D = 1 for univariate data)
inputs = Input(shape=(num_unrollings, D), name='lstm_input')

# First LSTM layer:
# Returns the full sequence of hidden states (one per time step) for the next LSTM layer to process
# Number of hidden units is specified in num_nodes[0]
x = LSTM(num_nodes[0], return_sequences=True)(inputs)
# Dropout layer to help prevent overfitting by randomly disabling a fraction of units
x = Dropout(dropout)(x)

# Second LSTM layer:
# Again returns the full sequence so the next LSTM layer receives the entire time series
x = LSTM(num_nodes[1], return_sequences=True)(x)
x = Dropout(dropout)(x)

# Third LSTM layer:
# This time we only return the last output in the sequence (i.e., a summary of the full input)
x = LSTM(num_nodes[2], return_sequences=False)(x)
x = Dropout(dropout)(x)

# Final output layer:
# A Dense layer with a single neuron to produce the final predicted value (e.g., next day's price)
outputs = Dense(1, name='price_output')(x)

# Define the full model by linking the input and output layers
model = Model(inputs=inputs, outputs=outputs, name='three_layer_lstm')

# Print a summary of the model structure
model.summary()

# Compile the model with:
# Mean Squared Error (MSE) as the loss function for regression
# Adam optimiser for efficient gradient descent
learning_rate = 0.001

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='mse',     # loss function to minimise
    metrics=['mse'] # also report MSE during training
)


### Model training

Here, we train an LSTM model to predict stock price movements over several training cycles (called *epochs*) and observe whether its predictions improve with each pass through the data. The goal is to see if the model can learn meaningful patterns in the time series and generalise to new, unseen points.

During training, we evaluate the model's performance not just on the training data, but also by making rolling forecasts at specific locations within the test set. These test points serve as reference positions where we ask the model to predict several steps ahead and then compare those predictions to the actual market data.

The prediction horizon defines how far into the future we want our model to forecast. In other words, it's the number of time steps the model is expected to predict beyond the current point. For example, if you're driving and your satnav tells you only the next 10 metres, it's not very helpful. But if it gives you directions 1 km ahead, you can plan your actions better. Similarly, the prediction horizon gives your model a chance to guide decisions further into the future.

The training process involves the following key steps:

- Select evaluation points (`test_points_seq`) across the time series where we want to test the model's forecasting ability.

- Then, for each *epoch*:
  - Unroll the data into short sequences of length `num_unrollings`.
  - Train the model on these sequences, one batch at a time, updating its internal weights.
  - Track the average loss on the training set to monitor progress.

- After training for that epoch:

  - For each test point:
    - Reset or update the model’s internal state.
    - Generate predictions step-by-step for `n_predict_once` time steps.
    - Compute the Mean Squared Error (MSE) between predicted values and the actual future stock prices.

The code then defines the required parameters (such as window length, prediction horizon, and batch size), and compiles the model with an appropriate optimiser and loss function:

In [ ]:
# Get the total number of time steps in the training dataset
train_seq_length = train_data.size  # Full length of the training data (used for iteration/tracking)

# Lists to store performance metrics and predictions across epochs
train_mse_ot = []             # Accumulate mean squared error on training set after each epoch
test_mse_ot = []              # Accumulate mean squared error on test set after each epoch
predictions_over_time = []   # Store the rolling predictions made at fixed test locations

n = 50 # Number of steps to predict

# Learning rate scheduler control: reduce learning rate if validation error stagnates
loss_nondecrease_threshold = 2  # Number of epochs to wait before reducing the learning rate

# Running average of training loss (used to monitor convergence)
average_loss = 0

# Define the region in the time series to evaluate model performance (have a look at previous plots for an interesting section of the timeseries).
test_start = 11000            # Starting index of the test window - you can choose any really.
test_end = 12000              # Ending index of the test window, likewise this is something you can decide on.

# Choose specific points at which to perform multi-step prediction
# This allows us to simulate how the model behaves at different stages of the time series
test_points_seq = np.arange(test_start, test_end, n).tolist()  # One test every n-steps

# A function to generate horizon-length predictions repeatedly across the test range
def rolling_forecasts(model,
                      series_scaled,          # The full, scaled time series (used as model input)
                      scaler,                 # The MinMaxScaler used to inverse-transform predictions
                      test_start=11000,       # Start of the test region
                      test_end=12000,         # End of the test region
                      window_size=50,         # Number of past steps the model uses to make a prediction
                      horizon=50):            # Number of future steps to predict at each test point

    # Define test points spaced by `horizon` (e.g., every 50 steps)
    test_points = np.arange(test_start, test_end, horizon)

    # Prepare output containers: x values (time indices) and y values (predictions)
    x_axis_seq, y_pred_seq = [], []

    # Loop through each test point to generate a rolling forecast
    for w in test_points:
        # Take a slice of the past window_size points before this test point
        seed = series_scaled[w - window_size : w].copy()

        # Array to hold horizon predictions (in scaled form)
        preds_scaled = np.empty(horizon)

        # Predict forward one step at a time, feeding predictions back into the model
        for t in range(horizon):
            # Predict the next value using the current seed window
            y_next = model.predict(seed.reshape(1, window_size, 1), verbose=0)[0, 0]

            # Save the prediction
            preds_scaled[t] = y_next

            # Shift the seed window and insert the predicted value at the end
            seed = np.roll(seed, -1)
            seed[-1] = y_next

        # Convert predictions back to original price scale using the inverse of the scaler
        preds_unscaled = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()

        # Record both the x-axis positions and the predicted values
        y_pred_seq.append(preds_unscaled)
        x_axis_seq.append(np.arange(w, w + horizon))

    # Return time indices and predictions
    return x_axis_seq, y_pred_seq


Now that we have constructed our model and ensured we are feeding the model with the data in the correct structure, we can proceed to put it all together and train the model.

We wrap everything into something called a TensorFlow dataset. This special format makes training more efficient. It allows us to group the data into batches (so the model learns from several sequences at once) and shuffle the order of training samples to help avoid overfitting. We also pre-load upcoming data behind the scenes, so the training loop runs smoothly without waiting for data to be prepared.

We then compile the model. This means specifying how it should learn. We use the Adam optimiser that adjusts how the model updates itself as it learns. We also use Mean Squared Error (MSE) as the metric to judge how far off the model's predictions are from the actual stock prices. The smaller the MSE, the better the model is doing.

To make learning even more efficient, we use a helpful technique called *ReduceLROnPlateau*. This watches the model's progress and, if the performance on the validation set stops improving, it gently lowers the learning rate. This makes the model fine-tune its behaviour more carefully as it nears the best solution, rather than taking big, disruptive steps.

Finally, we begin the actual training. We feed the model batches of sequences for several epochs, letting it adjust and improve each time. After every epoch, we check how it performs not only on the training data but also on a separate test set to see if it's generalising well to unseen patterns. All of this is reorded, giving us a detailed view of how well the model is learning over time:

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau  # callback to adjust learning rate when progress stalls

# Set the number of training cycles (epochs)
epochs = 15

# Prepare training and testing sequences using the sliding window approach
# Each input sequence has num_unrollings time steps, and the label is the next value in the sequence
X_train, y_train = make_windows(train_data, num_unrollings)
X_test,  y_test  = make_windows(test_data,  num_unrollings)

# Convert training data into a TensorFlow Dataset pipeline
train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))  # Wrap input-output pairs
      .shuffle(buffer_size=10000)                           # Randomise order to prevent overfitting to sequence order
      .batch(batch_size, drop_remainder=True)               # Group into batches; drop the last few if not divisible
      .prefetch(tf.data.AUTOTUNE)                           # Optimise performance by overlapping data loading and training
)

# Prepare test data similarly, but without shuffling (since we're evaluating, not training)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
      .batch(batch_size, drop_remainder=True)
      .prefetch(tf.data.AUTOTUNE)
)

# Compile the model with:
# Adam optimiser (adaptive learning rate)
# Mean Squared Error (MSE) as both the loss function and metric
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mse']
)

# Create a callback to reduce the learning rate when the model stops improving
# monitor='val_loss' watches the validation loss
# If it doesn't improve for 2 consecutive epochs (patience=2), the learning rate is halved (factor=0.5).
# But, it won't go below min_lr=1e-6
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1,
    min_lr=1e-6
)

# Train the model

history = model.fit(
    train_dataset,                  # Training data
    epochs=epochs,                  # Number of passes through the data
    validation_data=test_dataset,  # Evaluate model on test set after each epoch
    callbacks=[reduce_lr],         # Adjust learning rate automatically if needed
    verbose=1                       # Print progress after each epoch
)

### Evaluation
The training results show that the LSTM model has learned to forecast stock price trends with a good level of accuracy. During the first epoch, the training loss started relatively high, but the model quickly began to adjust, and by the second epoch, the training error had fallen sharply. This indicates that the model was able to capture meaningful patterns in the time series early on.

Over the course of 15 epochs, the training loss steadily decreased to below 0.001, and the validation loss followed a similar downward trend. The model consistently performed well on unseen test data, with validation mean squared error (MSE) remaining low - typically around 0.002 to 0.003 in the final few epochs. This suggests the model generalised well and wasn't simply memorising the training set.

The learning rate scheduler (`ReduceLROnPlateau`) played a role in fine-tuning. As soon as progress on the validation set began to plateau, the scheduler reduced the learning rate, allowing the model to make smaller, more refined updates. This adaptiveness helped the model stabilise and avoid overfitting, especially in later epochs when large updates might have disrupted the gains already made.

Overall, the model's learning curve was smooth and effective. With such low validation error on a normalised scale, it's clear that the network has captured useful temporal dynamics from the financial time series. Let's visualise these results to double check.

The first plot shows how the model's error changed over each epoch for both the training and validation sets. This helps us judge how well the model learned and whether it overfit or underfit:

In [ ]:
# Generate rolling forecasts using the trained model
# x_axis_seq: x-values for plotting each prediction window
# predictions_over_time: list of predicted sequences (each of length = horizon)
x_axis_seq, predictions_over_time = rolling_forecasts(
    model,
    series_scaled=all_mid_data_scaled,  # scaled full dataset
    scaler=scaler,                      # fitted MinMaxScaler to invert predictions later
    window_size=num_unrollings,        # how many past time steps the model uses
    horizon=num_unrollings             # how many steps ahead we forecast each time
)

# Plot training and validation loss curves to visualise learning progress over epochs
plt.figure(figsize=(15, 6))

# Plot mean squared error on the training data
plt.plot(history.history['loss'], label='Train MSE')

# Plot mean squared error on the validation data
plt.plot(history.history['val_loss'], label='Val MSE')

plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')

plt.legend()


The second plot visualises the model’s final 50-step-ahead forecast (unscaled back to actual price range), offering a concrete look at what the model expects after observing the last window of real stock data:

In [ ]:
# Plot the final multi-step forecast (last prediction made after training)
final_forecast = predictions_over_time[-1]  # Get the most recent forecast (last in the list)

plt.figure(figsize=(15, 6))

# Plot the unscaled forecast for the next n time steps
plt.plot(final_forecast)

plt.title("50‐Day forecast (Unscaled) from final epoch")
plt.xlabel("Steps ahead")

plt.ylabel("Price")

### Prediction
Now that our model is trained, we can explore its predictions by overlaying them on the original time series. Instead of focusing on exact values, we are more interested in the overall direction of the predicted trend, that is, whether prices are moving upwards or downwards over short horizons. This is often more practical than pinpointing the precise stock price, as it can guide strategic decisions such as whether to buy, hold, or sell.

To do this, we plot the original mid-price series, and then gradually overlay the predicted sequences in red. Each red line represents the model's forecast for the next n-days, based on what it had seen up to that point. By using a fading effect (increasing line transparency), we can visualise how predictions evolve over time: older forecasts appear faint, while newer predictions are shown more boldly.

This gives us a view of how the model perceives the trend at different points in the series. For example, consistent downward sloping forecasts might signal a bearish phase, suggesting it's not the best time to invest. Likewise, if the forecast tilts upward, it might indicate the start of a bullish trend:

In [ ]:
# Plot the true mid-price series (original smoothed and scaled back data)
plt.figure(figsize=(15, 6))

plt.plot(range(df.shape[0]), all_mid_data, color='b', lw=1)

# Set the starting transparency (alpha) value for the oldest predictions
start_alpha = 0.25

# Count how many prediction windows we generated from rolling forecasts
n_plots = len(x_axis_seq)

# Create a smooth range of alpha values from faint (older) to opaque (recent)
alphas = np.linspace(start_alpha, 1.0, n_plots) # Define the section of the time series we want to zoom in on for visualisation

# Overlay each forecast (xs = time steps, ys = predicted prices) on top of the true series
# Older forecasts appear more transparent; newer ones are bolder
for a, (xs, ys) in zip(alphas, zip(x_axis_seq, predictions_over_time)):
    plt.plot(xs, ys, color='r', alpha=a)  # plot each n-step prediction in red

plt.title('Rolling n-step LSTM forecasts')

plt.xlabel('Time (t)')
plt.ylabel('Mid-price ($)')

# Restrict the x-axis to the test range so we can zoom into the forecast period
start_test = 10700
end_test = 12200
plt.xlim(start_test, end_test)


We can see that the LSTM did not do better than the standard averaging.
We initialised the testing window with test data, predict the next point over that and create a new window with the next point. However, once we reach a point where the input window is fully composed of past predictions it terminates, moves forward one full window length, and resets the window with the true test data. We then start the whole process again.

This produces multiple trend-line like predictions over the test data, which we can visually inspect to see how well the model identified future trends.

We can see from the predicted trend lines that the network did appear to correctly predict the trends (and amplitude of trends) for a good portion of the time series.

Although not necessarily perfect, it does provide an example of how useful LSTM deep neural networks can be when applied to time series problems. If we were to careful tune the hyperparameters through experimentation, then we could certainly achieve better results.  I leave this to you as a challenge!

### What have we learnt?

Predicting stock price movements using machine learning is a challenging and often uncertain task. While our LSTM model shows that it can capture broad trends and short-term fluctuations to some extent, it is far from a crystal ball. Financial markets are influenced by countless external factors, including news, policy, and sentiment, which are not directly visible in the historical prices used by our model.

In our project, we observed that the LSTM was able to learn temporal patterns and generate reasonable multi-step forecasts. However, the accuracy of these forecasts depends heavily on the *hyperparameters* chosen. Key parameters like the learning rate, the number of LSTM layers, and the number of hidden units can affect the model's performance. Optimisers like Adam are commonly used and generally effective, but experimenting with different options is often worthwhile. Approaches such as *grid search* or *random search* can be helpful to systematically explore combinations of hyperparameters.

It's also worth remembering that we scaled our price data between 0 and 1, so predictions were not in absolute currency units. That's okay, what matters more is the shape and direction of the trend: whether the model anticipates an upward or downward movement, rather than the precise numerical value. This kind of trend forecasting can support investment decisions even without pinpointing exact prices.

However, LSTMs are not a panacea. One limitation is that they struggle with the non-stationary nature of financial data—market dynamics shift over time in ways that break assumptions made by many models. Emerging methods such as Bayesian deep learning offer more robust treatment of uncertainty in non-stationary environments. Likewise, newer architectures based on attention mechanisms, such as Transformers, have recently outperformed traditional RNNs like LSTM in many sequential tasks.

In short, LSTMs provide a useful entry point for time series forecasting and can reveal meaningful structure in noisy data. But more sophisticated models and evaluation strategies are essential if we want to approach the complexity of real-world financial forecasting.

### Next steps
We have also provided some additional stock market data for you to test, have a look in the <a href="https://github.com/martyn-harris-bbk/AppliedMachineLearning/tree/main/data" target="_blank">./data/</a> folder for more financial data sets (`iam.us.txt`, `bbc.us.txt`, and `aamc.us.txt`).  You could also download a non-finance dataset, for instance, a timeseries recording temperature or rainfall and attempt to model it.  You will need to make changes to the sliding windows and test data ranges as you may have more, or less data to work with than this example.